<a href="https://colab.research.google.com/github/tayyabaimam/Face_detection_opencv/blob/master/Credit_Card_Fraud_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [2]:
credit_card_data = pd.read_csv('/content/creditcard.csv')


In [3]:
#preprocessing
credit_card_data.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,0.090794,-0.551600,-0.617801,-0.991390,-0.311169,1.468177,-0.470401,0.207971,0.025791,0.403993,0.251412,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,-0.166974,1.612727,1.065235,0.489095,-0.143772,0.635558,0.463917,-0.114805,-0.183361,-0.145783,-0.069083,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,0.207643,0.624501,0.066084,0.717293,-0.165946,2.345865,-2.890083,1.109969,-0.121359,-2.261857,0.524980,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,-0.054952,-0.226487,0.178228,0.507757,-0.287924,-0.631418,-1.059647,-0.684093,1.965775,-1.232622,-0.208038,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,0.753074,-0.822843,0.538196,1.345852,-1.119670,0.175121,-0.451449,-0.237033,-0.038195,0.803487,0.408542,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


In [8]:
credit_card_data.tail()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
284802,172786.0,-11.881118,10.071785,-9.834783,-2.066656,-5.364473,-2.606837,-4.918215,7.305334,1.914428,4.356170,-1.593105,2.711941,-0.689256,4.626942,-0.924459,1.107641,1.991691,0.510632,-0.682920,1.475829,0.213454,0.111864,1.014480,-0.509348,1.436807,0.250034,0.943651,0.823731,0.77,0
284803,172787.0,-0.732789,-0.055080,2.035030,-0.738589,0.868229,1.058415,0.024330,0.294869,0.584800,-0.975926,-0.150189,0.915802,1.214756,-0.675143,1.164931,-0.711757,-0.025693,-1.221179,-1.545556,0.059616,0.214205,0.924384,0.012463,-1.016226,-0.606624,-0.395255,0.068472,-0.053527,24.79,0
284804,172788.0,1.919565,-0.301254,-3.249640,-0.557828,2.630515,3.031260,-0.296827,0.708417,0.432454,-0.484782,0.411614,0.063119,-0.183699,-0.510602,1.329284,0.140716,0.313502,0.395652,-0.577252,0.001396,0.232045,0.578229,-0.037501,0.640134,0.265745,-0.087371,0.004455,-0.026561,67.88,0
284805,172788.0,-0.240440,0.530483,0.702510,0.689799,-0.377961,0.623708,-0.686180,0.679145,0.392087,-0.399126,-1.933849,-0.962886,-1.042082,0.449624,1.962563,-0.608577,0.509928,1.113981,2.897849,0.127434,0.265245,0.800049,-0.163298,0.123205,-0.569159,0.546668,0.108821,0.104533,10.00,0
284806,172792.0,-0.533413,-0.189733,0.703337,-0.506271,-0.012546,-0.649617,1.577006,-0.414650,0.486180,-0.915427,-1.040458,-0.031513,-0.188093,-0.084316,0.041333,-0.302620,-0.660377,0.167430,-0.256117,0.382948,0.261057,0.643078,0.376777,0.008797,-0.473649,-0.818267,-0.002415,0.013649,217.00,0


In [7]:
credit_card_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 284807 entries, 0 to 284806
Data columns (total 31 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   Time    284807 non-null  float64
 1   V1      284807 non-null  float64
 2   V2      284807 non-null  float64
 3   V3      284807 non-null  float64
 4   V4      284807 non-null  float64
 5   V5      284807 non-null  float64
 6   V6      284807 non-null  float64
 7   V7      284807 non-null  float64
 8   V8      284807 non-null  float64
 9   V9      284807 non-null  float64
 10  V10     284807 non-null  float64
 11  V11     284807 non-null  float64
 12  V12     284807 non-null  float64
 13  V13     284807 non-null  float64
 14  V14     284807 non-null  float64
 15  V15     284807 non-null  float64
 16  V16     284807 non-null  float64
 17  V17     284807 non-null  float64
 18  V18     284807 non-null  float64
 19  V19     284807 non-null  float64
 20  V20     284807 non-null  float64
 21  V21     28

In [9]:
credit_card_data.isnull().sum()

,0
Time,0
V1,0
V2,0
V3,0
V4,0
V5,0
V6,0
V7,0
V8,0
V9,0


In [10]:
#counting variations of values of a certain column
credit_card_data['Class'].value_counts()

,count
Class,
0,284315
1,492


In [11]:
#seperating the data for analysis
legit = credit_card_data[credit_card_data.Class == 0]
fraud = credit_card_data[credit_card_data.Class == 1]

In [12]:
#visualizing data
print(legit.shape)
print(fraud.shape)#

(284315, 31)
(492, 31)


In [14]:
legit.Amount.describe()

,Amount
count,284315.000000
mean,88.291022
std,250.105092
min,0.000000
25%,5.650000
50%,22.000000
75%,77.050000
max,25691.160000


In [15]:
fraud.Amount.describe()

,Amount
count,492.000000
mean,122.211321
std,256.683288
min,0.000000
25%,1.000000
50%,9.250000
75%,105.890000
max,2125.870000


In [16]:
#observing diff btw fraud and legit transactions
credit_card_data.groupby('Class').mean()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount
Class,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,94838.202258,0.008258,-0.006271,0.012171,-0.007860,0.005453,0.002419,0.009637,-0.000987,0.004467,0.009824,-0.006576,0.010832,0.000189,0.012064,0.000161,0.007164,0.011535,0.003887,-0.001178,-0.000644,-0.001235,-0.000024,0.000070,0.000182,-0.000072,-0.000089,-0.000295,-0.000131,88.291022
1,80746.806911,-4.771948,3.623778,-7.033281,4.542029,-3.151225,-1.397737,-5.568731,0.570636,-2.581123,-5.676883,3.800173,-6.259393,-0.109334,-6.971723,-0.092929,-4.139946,-6.665836,-2.246308,0.680659,0.372319,0.713588,0.014049,-0.040308,-0.105130,0.041449,0.051648,0.170575,0.075667,122.211321


In [17]:
#undersampling to use same amount of both types of data
legit_sample = legit.sample(n=492)

In [18]:
#concatenating two dfs
new_dataset= pd.concat([legit_sample, fraud], axis=0)

In [21]:
new_dataset.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
94660,64975.0,-0.476052,0.775520,0.920655,0.679705,0.523202,-0.320640,0.961575,-0.263727,-0.046640,0.332006,-1.399525,-1.245293,-1.254415,0.290793,1.291003,-0.098490,-0.534567,0.524093,0.744741,0.107873,-0.011255,0.258958,-0.104399,-0.482020,-0.123212,-0.294662,0.141177,-0.027501,38.99,0
281611,170302.0,-5.034683,-12.155822,-4.527769,2.219104,-4.375383,0.380424,4.385223,-1.140605,0.892836,-2.145123,-0.351511,1.237677,1.918829,0.606443,1.550776,0.680295,-0.420118,0.241819,-0.960592,7.358635,2.058558,-2.221445,-2.833333,0.145944,-1.957437,-0.052993,-0.720871,0.593201,3670.00,0
240708,150713.0,-2.325852,1.058483,-0.947228,-0.363285,0.813319,-1.996398,0.609785,0.608481,-0.588264,-2.684228,-0.537592,-1.131388,-1.498974,-1.765864,0.634708,1.298336,1.965452,1.637657,-1.214203,-0.381023,0.167583,0.157850,-0.266568,-0.204675,1.185322,-0.059075,-0.213995,-0.341232,1.00,0
52508,45493.0,-1.843078,2.039961,0.647949,-0.217428,-0.532802,-0.524398,-0.069993,0.593072,-0.339765,-0.068044,-0.895643,0.834727,1.780355,0.073894,0.766549,0.472887,-0.447856,-0.089900,0.094892,-0.027464,0.052535,-0.004035,-0.053341,-0.075723,-0.241697,0.092772,-1.050917,-0.446669,1.99,0
196842,131720.0,0.182754,1.560557,-0.041571,4.174957,1.471830,1.020713,0.892271,0.050091,-2.284900,1.874116,-0.190540,-0.602223,-0.723141,0.776439,-0.768926,-0.406685,-0.000960,0.490314,1.916558,0.071806,0.145617,0.464307,-0.127622,0.190776,-0.802906,0.296769,0.105620,0.218573,9.08,0


In [22]:
new_dataset['Class'].value_counts()

,count
Class,
0,492
1,492


In [23]:
new_dataset.groupby('Class').mean()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount
Class,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,94787.995935,-0.020787,0.068444,0.040702,0.108796,0.127807,0.020569,-0.042857,0.024008,-0.013269,0.095889,0.013914,0.040265,0.058340,-0.118196,0.028434,0.029521,0.042784,0.000498,0.035521,-0.005914,0.004233,-0.031734,0.003073,0.003671,-0.028373,-0.022453,0.027183,0.004767,74.364350
1,80746.806911,-4.771948,3.623778,-7.033281,4.542029,-3.151225,-1.397737,-5.568731,0.570636,-2.581123,-5.676883,3.800173,-6.259393,-0.109334,-6.971723,-0.092929,-4.139946,-6.665836,-2.246308,0.680659,0.372319,0.713588,0.014049,-0.040308,-0.105130,0.041449,0.051648,0.170575,0.075667,122.211321


In [24]:
#splitting into features and target
X = new_dataset.drop(columns='Class', axis=1)
Y = new_dataset['Class']

In [25]:
print(X)

            Time        V1         V2        V3  ...       V26       V27       V28   Amount
94660    64975.0 -0.476052   0.775520  0.920655  ... -0.294662  0.141177 -0.027501    38.99
281611  170302.0 -5.034683 -12.155822 -4.527769  ... -0.052993 -0.720871  0.593201  3670.00
240708  150713.0 -2.325852   1.058483 -0.947228  ... -0.059075 -0.213995 -0.341232     1.00
52508    45493.0 -1.843078   2.039961  0.647949  ...  0.092772 -1.050917 -0.446669     1.99
196842  131720.0  0.182754   1.560557 -0.041571  ...  0.296769  0.105620  0.218573     9.08
...          ...       ...        ...       ...  ...       ...       ...       ...      ...
279863  169142.0 -1.927883   1.125653 -4.518331  ...  0.788395  0.292680  0.147968   390.00
280143  169347.0  1.378559   1.289381 -5.004247  ...  0.739467  0.389152  0.186637     0.76
280149  169351.0 -0.676143   1.126366 -2.213700  ...  0.471111  0.385107  0.194361    77.89
281144  169966.0 -3.113832   0.585864 -5.399730  ...  0.606116  0.884876 -0.2537

In [26]:
print(Y)

94660     0
281611    0
240708    0
52508     0
196842    0
         ..
279863    1
280143    1
280149    1
281144    1
281674    1
Name: Class, Length: 984, dtype: int64


In [34]:
#splitting data into train and test set
X_train,X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, stratify=Y, random_state=2)

In [35]:
print(X.shape, X_train.shape, X_test.shape)

(984, 30) (787, 30) (197, 30)


In [46]:
#logistic regression
model = LogisticRegression()

In [47]:
#training
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)
model.fit(X_train_scaled, Y_train)

LogisticRegression()

In [48]:
x_train_prediction = model.predict(X_train_scaled)
training_data_accuracy = accuracy_score(x_train_prediction, Y_train)

In [50]:
print('Accuracy on Training data : ', training_data_accuracy)

Accuracy on Training data :  0.951715374841169


In [51]:
#accuracy on test data
x_test_prediction= model.predict(X_test_scaled)
test_data_accuracy=accuracy_score(x_test_prediction, Y_test)

In [52]:
print('Accuracy score on Test Data : ', test_data_accuracy)

Accuracy score on Test Data :  0.9238578680203046
